## FNOP - Google Universal Sentance Encoder + KNN Experiment

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, LabelEncoder

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
import nltk
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

In [6]:
from sklearn.model_selection import train_test_split

### Read data and grab random sample

Uses traiing data filtered to top 40 parts

In [7]:
#df_tmp = pd.read_csv('C:/Projects/FCSD/FNOP/old/Traindata_pilot_sample_v0.csv', engine='python')
df_tmp = pd.read_csv('C:/Users/lguerra5/Documents/FNOP/data/Traindata_pilot_sample_v0_top40parts.csv', engine='python')

In [8]:
np.random.seed(47)
df = df_tmp.sample(n=10000)
df = df[['cust_txt', 'target']]

In [12]:
### create count by target df
#target_count_df = df_1.groupby('target')['cust_txt'].count().to_frame(name='row_count').reset_index()

### Process Customer Input Text (from Jay's code)

In [10]:
lookup_table = 'C:/Users/lguerra5/Documents/FNOP/data/Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'
syn_df=pd.read_excel(lookup_table, engine='openpyxl')
syn_df = syn_df[syn_df['Review'] != "D"]
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [11]:
def TextPreprocess(text):
        # Avoid mixed data type errors and apply str on all rows
        # convert to lower case
        text = str(text).lower()

        #Remove HTML Tags
        text = re.sub('<[^>]*>', '', text)
        r1 = re.compile('<.*?>')
        text = re.sub(r1, ' ', text)

#         # Remove any token with numbers and/or special characters
#         r2 = re.compile(r'^[0-9\W\_]+$')
#         text = re.sub(r2, ' ', text)
        
        # move parenthesis content to the end of paragraph
        paren_content_extract = re.compile(r'\(([^()]+)\)')
        p_cont = paren_content_extract.findall(text)
        text = re.sub(r'\([^()]*\)', ' ', text)
        text = text + '. '.join(p_cont) + '.'
        
        # hard code to convert abbreviations with \'s
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", "have not ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r"\'scuse", " excuse ", text)
        
        # if a token starts with numerical or # sign, remove it
        text = re.sub('\b[\d#]+\S*\b', ' ', text)
        # if a token ends with numberical, remove it
        text = re.sub(r'\b\S+\d+\b', ' ', text)

#         #Removing Punctuations
#         text = re.sub(r'[?|!|\'|"|#]',r'',text)
#         text = re.sub(r'[.|,|)|(|\|/]',r' ',text)     

        #Removing Punctuations
        text = re.sub(r'[\'|"|#]',r'',text)
        text = re.sub(r'[)|(]',r' ',text) 
        
        # whitespace characters are replaced into space, multiple dots replaced into single dot
        text = re.sub('\s+', ' ', text)
        text = re.sub(r'\.+', '.', text)

        # Tokenize all text data using nltk package
        text = word_tokenize(text)

#         # Remove stop words
#         stop = stopwords.words('english')
#         stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', \
#                  'c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', \
#                  'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

#         # Combine default nltk stop list and custom stop list to generate a deduplicated list
#         english_stops = list(set(stop+stop1))

#         text = [item for item in text if item not in english_stops]

#         # Remove special characters occuring as prefixes
#         text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
#         # Remove special characters occuring as suffixes
#         text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

#         # Replace special characters with spaces when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length
#         text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

#         # Replace special characters with null when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length
#         text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]

#         # Remove stop words after data cleansing steps to remove any residual stop words
#         text = [item for item in text if item not in english_stops]

#         # remove any numbers after data cleansing steps to remove any residual numeric values
#         r3 = re.compile(r'^[0-9\W\_]+$')
#         text = [i for i in text if not r3.match(i)]

        #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
        #spell=SpellChecker() 
        text = [syn_dict.get(str(i), str(i)) for i in text]

        # Convert the list to string before passing to lemmatization module
        text = ' '.join(text)

#         # Perform Lemmatization on the cleaned up data before tfidf vectorization
#         text = lemmatize_with_postag(text)
        text = text.replace(' .', '.').replace(' ?', '?').replace(' !', '!').replace(' ,', ',').replace(' /', '/')
        return(text)

In [12]:
df['process_txt']=df['cust_txt'].apply(lambda x: TextPreprocess(x))

In [13]:
df = df.drop("cust_txt", axis=1)

### Use Google Universal sentence encoder to convert text into vectors

In [14]:
### load the model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [15]:
sentence_embeddings_X = model(df['process_txt']).numpy()

In [16]:
sentence_embeddings_X.shape

(10000, 512)

In [17]:
sentence_embeddings_X

array([[ 0.04594347,  0.07155082, -0.01262424, ...,  0.00998854,
        -0.00268265, -0.00214981],
       [ 0.04314676,  0.0345182 ,  0.04794639, ...,  0.00219908,
        -0.03629711,  0.01147657],
       [ 0.05486036,  0.04175026,  0.05406369, ..., -0.00662801,
        -0.00158086,  0.0074383 ],
       ...,
       [ 0.01528234,  0.05599257,  0.03571864, ...,  0.05302623,
         0.0226225 ,  0.00758379],
       [-0.01126657,  0.08220697,  0.03660792, ..., -0.01829402,
         0.00545433,  0.04418008],
       [-0.03583535,  0.06916902, -0.0608791 , ...,  0.03137726,
        -0.00139166,  0.04870439]], dtype=float32)

### KNN w/ Universal Sentence Encoder

In [19]:
from sklearn import metrics

In [18]:
# Split data for model
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings_X, df['target'], test_size=0.2, random_state=42)

In [20]:
k = 1
knn = KNeighborsClassifier(n_neighbors = k, metric = 'cosine')
knn.fit(X_train,y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=1)

In [23]:
y_pred = knn.predict(X_test)
y_pred[0:5]

array(['3504', '8501', '18125', 'other1', '14B291'], dtype=object)

In [27]:
y_test[0:5]

246082      3504
270577      8501
115516     18125
330213    other1
149        10346
Name: target, dtype: object

In [22]:
print("Train set Accuracy: ", metrics.accuracy_score(y_train, knn.predict(X_train)))

Train set Accuracy:  0.98575


In [24]:
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_pred))

Test set Accuracy:  0.6005


In [26]:
error = np.mean(y_pred != y_test)
error

0.3995

In [33]:
print(metrics.confusion_matrix(y_test, y_pred))

[[19  3  0 ...  2  4  0]
 [ 3 10  1 ...  0  2  0]
 [ 2  0 74 ...  0  1  2]
 ...
 [ 0  0  0 ... 18  0  2]
 [ 2  3  1 ...  3 22  6]
 [ 2  3  0 ...  1  4  2]]


In [34]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       10346       0.58      0.51      0.54        37
       11002       0.34      0.40      0.37        25
        1104       0.58      0.58      0.58       127
        1125       0.73      0.71      0.72       104
       12029       0.24      0.30      0.27        37
       13008       0.79      0.61      0.69        31
       13404       0.68      0.66      0.67        29
       14529       0.67      0.67      0.67        43
      14A626       0.94      0.85      0.89        20
      14B291       0.57      0.47      0.52        51
      15K601       0.83      0.91      0.87        96
       17508       0.85      0.91      0.88        32
       18125       0.58      0.69      0.63        61
       18808       0.88      0.95      0.91       110
       19703       0.41      0.45      0.43        73
       19712       0.05      0.06      0.05        16
       19805       0.57      0.58      0.57        50
       19860       0.08    